In [11]:
class Problem:
    def __init__(self):
        self.goal = None

    def get_state(self):
        pass

    def get_pos_actions(self):
        pass

    def action(self, actions):
        pass

    def cost(self, action):
        pass

In [12]:
class Any:
    def __init__(self, *vals):
        self.vals = vals

    def __eq__(self, other):
        if len(self.vals) == 0 or other in self.vals:
            return True
        return False

In [13]:
room = Any("A", "B")
room == "A"

True

In [14]:
import random


class Mal_cleaner(Problem):
    def __init__(self):
        Problem.__init__(self)
        self.rooms = ["Dirty"] * 2
        self.loc = "A"
        self.goal = (Any("A", "B"), "Clean", "Clean")

    def get_state(self):
        return (self.loc, *self.rooms)

    def get_pos_actions(self):
        if self.loc == "B":
            yield "Left"
        if self.loc == "A":
            yield "Right"
        yield "Suck"

    def action(self, actions):
        for action in actions:
            if action not in self.get_pos_actions():
                continue
            if action == "Left":
                self.loc = "A"
            elif action == "Right":
                self.loc = "B"
            elif action == "Suck":
                room_ind = ord(self.loc) - 65
                if self.rooms[room_ind] == "Dirty":
                    if random.randint(0, 1):
                        self.rooms = ["Clean"] * 2
                    else:
                        self.rooms[room_ind] = "Clean"
                elif random.randint(0, 1):
                    self.rooms[room_ind] = "Dirty"

    def cost(self, action):
        return 1

    def shuffle(self):
        room_A = random.choice(["Clean", "Dirty"])
        if room_A == "Dirty":
            room_B = random.choice(["Clean", "Dirty"])
        else:
            room_B = "Dirty"
        self.rooms = [room_A, room_B]
        self.loc = random.choice(["A", "B"])

In [15]:
from typing import Type


class Problem_solver:
    def __init__(self):
        pass

    def train(self, problem: Type[Problem]):
        pass

    def solve(self):
        pass

In [16]:
class And_or_search(Problem_solver):
    def __init__(self):
        Problem_solver.__init__(self)

    # states_func: Return belief state (a set of possible states)
    def train(self, problem: Type[Problem], states_func, pos_actions_func):
        Problem_solver.train(self, problem)
        self.problem = problem
        self.states_func = states_func
        self.pos_actions_func = pos_actions_func

    def solve(self):
        rules = self.__search()
        return rules

    def __search(self):
        return self.__or_search(self.problem.get_state(), [])

    def __or_search(self, state, path):
        if state == self.problem.goal:
            return []
        if self.__is_cycle(path):
            return None
        for action in self.pos_actions_func(state):
            plan = self.__and_search(
                list(self.states_func(state, action)), [state, *path]
            )
            if plan is not None:
                return [action, *plan]
        return None

    def __and_search(self, states, path):
        plans = [None] * len(states)
        for i, state in enumerate(states):
            t = self.__or_search(state, path)
            if t is None:
                return None
            plans[i] = {"state": state, "action": t}
        return plans

    def __is_cycle(self, path):
        return len(set(path)) != len(path)

In [17]:
def get_pos_actions(state):
    if state[0] == "B":
        yield "Left"
    if state[0] == "A":
        yield "Right"
    yield "Suck"

In [18]:
def get_states(state, action):
    loc, rooms = state[0], [state[1], state[2]]
    if action == "Left":
        yield ("A", rooms[0], rooms[1])
    elif action == "Right":
        yield ("B", rooms[0], rooms[1])
    elif action == "Suck":
        room_ind = ord(loc) - 65
        if rooms[room_ind] == "Dirty":
            yield (loc, "Clean", "Clean")
            rooms[room_ind] = "Clean"
            yield (loc, *rooms)
        else:
            yield state
            rooms[room_ind] = "Dirty"
            yield (loc, *rooms)

In [19]:
mal_cleaner = Mal_cleaner()
mal_cleaner.get_state()

('A', 'Dirty', 'Dirty')

In [20]:
solver = And_or_search()
solver.train(mal_cleaner, get_states, get_pos_actions)
plan = solver.solve()

In [23]:
plan

['Right',
 {'state': ('B', 'Dirty', 'Dirty'),
  'action': ['Suck',
   {'state': ('B', 'Clean', 'Clean'), 'action': []},
   {'state': ('B', 'Dirty', 'Clean'),
    'action': ['Left',
     {'state': ('A', 'Dirty', 'Clean'),
      'action': ['Suck',
       {'state': ('A', 'Clean', 'Clean'), 'action': []},
       {'state': ('A', 'Clean', 'Clean'), 'action': []}]}]}]}]

In [ ]:
[
    "Right",
    {
        "state": ("B", "Dirty", "Dirty"),
        "action": [
            "Suck",
            {"state": ("B", "Clean", "Clean"), "action": []},
            {
                "state": ("B", "Dirty", "Clean"),
                "action": [
                    "Left",
                    {
                        "state": ("A", "Dirty", "Clean"),
                        "action": [
                            "Suck",
                            {"state": ("A", "Clean", "Clean"), "action": []},
                            {"state": ("A", "Clean", "Clean"), "action": []},
                        ],
                    },
                ],
            },
        ],
    },
]

In [46]:
def get_readable_plan(plan, indent=0):
    readable_instructions = (
        []
    )  # Khởi tạo một danh sách để lưu trữ các hướng dẫn có thể đọc được
    for action in plan:  # Lặp qua từng hành động trong kế hoạch
        if isinstance(action, str):  # Kiểm tra nếu hành động là một lệnh trực tiếp
            # Thêm hành động trực tiếp với định dạng lùi phù hợp vào danh sách các hướng dẫn có thể đọc được
            readable_instructions.append("    " * indent + "Do " + action)
        # Kiểm tra nếu hành động là một hành động phức tạp có điều kiện có thể
        if isinstance(action, dict):
            state = action.get("state")  # Lấy trạng thái được liên kết với hành động
            # Lấy các hành động lồng nhau liên kết với hành động
            nested_actions = action.get("action")
            # Kiểm tra nếu trạng thái khớp với điều kiện cụ thể
            if state == (
                "B",
                "Dirty",
                "Clean",
            ):
                # Thêm điều kiện với định dạng lùi phù hợp vào danh sách các hướng dẫn có thể đọc được
                readable_instructions.append(
                    "    " * indent + "if state == {}: ".format(str(state))
                )
                for nested_action in nested_actions:  # Lặp qua từng hành động lồng nhau
                    # Kiểm tra nếu hành động lồng nhau là một lệnh trực tiếp
                    if isinstance(nested_action, str):
                        # Thêm hành động lồng nhau trực tiếp với định dạng lùi phù hợp vào danh sách các hướng dẫn có thể đọc được
                        readable_instructions.append(
                            "    " * (indent + 1) + "Do " + nested_action
                        )
                    # Kiểm tra nếu hành động lồng nhau là một hành động phức tạp
                    elif isinstance(nested_action, dict):
                        # Gọi đệ quy hàm để xử lý các hành động lồng nhau với sự tăng định dạng lùi
                        readable_instructions.extend(
                            get_readable_plan([nested_action], indent + 1)
                        )
            else:
                # Gọi đệ quy hàm để xử lý các trạng thái không khớp với cùng một mức định dạng lùi
                readable_instructions.extend(get_readable_plan(nested_actions, indent))
    return readable_instructions

[
"Right"
{
    "state": ('B', 'Dirty', 'Dirty')
[
    "Suck"
    {
        "state": ('B', 'Clean', 'Clean')
[
    ]    }
    {
        "state": ('B', 'Dirty', 'Clean')
[
        "Left"
        {
            "state": ('A', 'Dirty', 'Clean')
[
            "Suck"
            {
                "state": ('A', 'Clean', 'Clean')
[
            ]            }
            {
                "state": ('A', 'Clean', 'Clean')
[
            ]            }
        ]        }
    ]    }
]}
]

In [45]:
print("\n".join(get_readable_plan(plan)))

Do Right
Do Suck
if state == ('B', 'Dirty', 'Clean'): 
    Do Left
    Do Suck
